# Neuron Skeletonization Service Notebook

Copyright (c) 2025 Open Brain Institute

+ Author(s): 
    - Michael W. Reimann < michael.reimann@openbraininstitute.org >
    - Marwan Abdellah < marwan.abdellah@openbraininstitute.org >

Last modified: 11.2025

## Imports and setting up platform authentication

We begin by importing required packages.

In [ ]:
# All essential imports needed for the notebook
import httpx
import os
import time 
import json 
import pathlib
from uuid import UUID
from urllib.parse import urlparse
from IPython.display import display, HTML
from obi_auth import get_token
from obi_notebook import get_projects, get_entities, get_environment
from entitysdk.client import Client
from entitysdk.models import CellMorphology, EMCellMesh
from entitysdk.models.asset import AssetLabel

## Authentication and project selection
We authenticate with the OBI platform.
### Project selection
Select from the dropdown menu the project that the output (a morphology with extracted spines) should be registered to. It will be available only in that project context. 

The widget lists all projects you have access to.

In [ ]:
environment = get_environment.get_environment()
token = get_token(environment=environment, auth_mode="daf")
project_context = get_projects.get_projects(token)

## Set up clients

With the information provided above we assemble a client that can interact with the entity database.

In [ ]:
# Initialize the client and search for EMCellMesh entities
client = Client(environment=environment, token_manager=token, project_context=project_context)
entitycore_api_url = urlparse(client.api_url)
platform_base_url = f"{entitycore_api_url.scheme}://{entitycore_api_url.netloc}"
mesh_api_base_url = f"{platform_base_url}/api/small-scale-simulator/mesh/skeletonization"

http_client = httpx.Client()

mesh_api_headers = httpx.Headers({
  "Authorization": f"Bearer {token}",
  "virtual-lab-id": str(project_context.virtual_lab_id),
  "project-id": str(project_context.project_id)
})

## Display EMCellMesh table

Input into the skeletonization process is a mesh describing the surface of a neuron. 

Below we will display a list of all surface meshes you have access to in the project context you selected above. This is a combination of publicly available entities open to all users and cell meshes you or your team have registered inside the project.
You can filter the list based on the EM dataset the meshes originate from. You can also search for specific neuron identifiers (typically called "pt_root_id").

Please select one mesh from the list below.

In [ ]:
mesh_ids = []
mesh_ids = get_entities.get_entities(
    'em-cell-mesh', token, [], env=environment, project_context=project_context, 
    multi_select=False, page_size=20, add_columns=["dense_reconstruction_cell_id"])

## Check the selection 

In [ ]:
if len(mesh_ids) == 0:
    display(HTML("<h3 style='color:#EDB95E'>⚠️ No mesh selected</h3>"))
    raise SystemExit()
mesh_id = mesh_ids[0]
input_entity = client.get_entity(entity_id=mesh_id, entity_type=EMCellMesh)

## Prepare task params

We submit the skeletonization and spine extraction as a task to a separate computing system. Here, we assemble the specifications.

In [ ]:
input_params = {
  "name": mesh_id,
  "description": f"Reconstructed morphology and extracted spines of neuron {input_entity.dense_reconstruction_cell_id}."
}

skeletonization_params = {
  "em_cell_mesh_id": mesh_id,
  "neuron_voxel_size": 0.1, # in microns
  "spines_voxel_size": 0.05, # in microns
  "segment_spines": True
}

## Submit mesh skeletonization task

In [ ]:
start_res = http_client.post(
  f"{mesh_api_base_url}/run",
  params=skeletonization_params,
  headers=mesh_api_headers,
  json=input_params
)

job_id = None

if start_res.is_success:
  job_id = start_res.json().get("id")
else:
  print(start_res.text)
  raise RuntimeError("Failed to submit mesh skeletonization task")

## Wait for the job to complete

In [ ]:
output_morphology_id = None
prev_status = None

while True:
  status_res = http_client.get(
    f"{mesh_api_base_url}/jobs/{job_id}",
    headers=mesh_api_headers
  )

  if not status_res.is_success:
    print(status_res.text)
    raise RuntimeError("Failed to get job status")

  job = status_res.json()
  status = job.get('status')

  if status != prev_status:
    print(f"{time.strftime("%H:%M:%S", time.localtime())}  Status: {status}")
    prev_status = status

  if status == 'finished':
    output_morphology_id = UUID(job.get('output').get('morphology').get('id'))
    break
  elif status == 'failed':
    print(json.dumps(job, indent=2))
    raise RuntimeError("Skeletonization failed")

  time.sleep(15)

## Get the resulting registered morphology 

In [ ]:
cell_morphology = client.get_entity(output_morphology_id, entity_type=CellMorphology)
asset = next((asset for asset in cell_morphology.assets if asset.label == AssetLabel.morphology_with_spines), None)

# Download the file
client.download_assets(cell_morphology,selection={"label": AssetLabel.morphology_with_spines}, output_path=pathlib.Path(os.getcwd())).one()